In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
df = pd.read_csv("current_netcdfs/PRJC1.txt", r" +")
# units row, drop
df = df.drop(0)
df = df.drop(df[df["WDIR"] == "MM"].index)
df

In [ ]:
# notes: wind direction is where wind is coming from, pointing in opposite direction
# 3% of wspd added to ocean vector
uw_obs = -df['WSPD'].astype(float).values * np.sin(np.deg2rad(df['WDIR'].astype(int).values))
vw_obs = -df['WSPD'].astype(float).values * np.cos(np.deg2rad(df['WDIR'].astype(int).values))

In [ ]:
timestamp = df["#YY"] + "-" + df["MM"] + "-" + df["DD"] + "T" + df["hh"] + ":" + df["mm"]
timestamp = pd.to_datetime(timestamp)

In [ ]:
new_df = pd.DataFrame({"time": timestamp, "u": uw_obs, "v": vw_obs})
new_df = new_df.sort_values(by="time")
new_df

In [ ]:
u = xr.DataArray(new_df["u"], coords={"time": new_df["time"]})
v = xr.DataArray(new_df["v"], coords={"time": new_df["time"]})

In [ ]:
ds = xr.Dataset({"U": u, "V": v})
ds

In [ ]:
ds.to_netcdf("current_netcdfs/Hunington_wind.nc")